In this file, a lstm model is used <br>
Rest of the code for data input is same as in previous model

Code for setting up the LSTM parameters

n_input: dimension of X <br>
n_steps: the number of time steps (stages)<br>
n_hidden: the number of hidden units<br>
num_layers: the number of LSTM layers<br>
n_classes: dimension of Y<br>

In [ ]:
n_input = p
n_steps = 10
n_hidden = 200  # should be more than the number of input units
num_layers = 2
n_classes = p

Setting the dimension for the input and output data (placeholders)

In [ ]:
x_vals = tf.placeholder("float", [None, n_steps, n_input])
y_vals = tf.placeholder("float", [None, n_classes])

Initializing variables

In [ ]:
weights = tf.Variable(tf.random_uniform(shape=[n_hidden, n_classes],maxval=tf.sqrt(6/(n_hidden+n_classes+0.0)),minval=-tf.sqrt(6/(n_hidden+n_classes+0.0))))
biases = tf.Variable(tf.constant([0.0]*n_classes))

Defining the RNN layers and model <br>
Dropouts and forget_bias is also set


In [ ]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    cell = tf.contrib.rnn.MultiRNNCell([rnn.DropoutWrapper(rnn.LSTMCell(n_hidden, forget_bias = 1.0), input_keep_prob = 0.8, output_keep_prob = 0.8) for _ in range(num_layers)])

    # Get lstm cell output
    outputs, states = rnn.static_rnn(cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights) + biases

pred = RNN(x, weights, biases)

The following code provides gradient clippingto check gradient explosion

In [ ]:
optimizer = tf.train.AdamOptimizer(learningRate)
gradients, variables = zip(*optimizer.compute_gradients(loss))
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
optimize = optimizer.apply_gradients(zip(gradients, variables))
optimizer = optimizer.minimize(loss)

Tensorflow code for session and variables initialization

In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

Running the model and evaluating the graph

In [ ]:
# Launch the graph

with tf.Session() as sess:
    # session initialization is not required when using the restored values
    
    sess.run(init)
    #saver.restore(sess, "/home/ashutosh/Desktop/instacart/model.ckpt")
    step = 1
    epoch = 1

    # Keep training until reach max iterations
    while step < training_iters:

        if (step+1)%training_iters/10 == 0:
            epoch += 0.5
            rate = rate/epoch

        batch_size = int(np.random.uniform(16, 32, 1))
        rand_index = np.random.choice(len(x_vals_train), size=batch_size)

        batch_x = x_vals_train[rand_index]
        batch_y = y_vals_train[rand_index]

        batch_x = batch_x.reshape((batch_size, n_steps, n_input))

        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, learningRate : rate})

        # Get and store the train loss
        train_temp_loss = sess.run(loss, feed_dict={x: x_vals_train, y: y_vals_train, learningRate:rate})
        train_loss.append(train_temp_loss)

        # Get and store the test loss
        test_temp_loss = sess.run(loss, feed_dict={x: x_vals_test, y: y_vals_test})
        test_loss.append(test_temp_loss)